이 프로젝트 파일에서는 '어떤 전략을 따라야 바보가 주식시장에서 성공할까'라는 주제를 다룬다.
우리는 '다른 것은 하나도 모르고, 단지 하나의 전략에만 따르는 바보'가 있다고 가정하고,
각 전략에 따른 수익률을 바탕으로 어떤 전략을 따라야 하는지를 파악한다.


# 1. 절대 모멘텀 전략


- 절대 모멘텀 전략은 퀀트 투자 전략 중 하나이다.(퀀트 투자:데이터 분석을 기반으로 하는 투자 전략)

- 절대 모멘텀 전략은 듀얼 모멘텀 전략 중 하나이다. 투자 자산 가운데 상대적으로 상승 추세가 강한 종목에 투자하는 상대 모멘텀 전략과, 절대 모멘텀 전략이 있다.

- 최근 n개월간 수익률이 양수이면 매수하고 음수이면 매도하는 long only 포지션으로 절대 모멘텀 전략을 만들어보자.

- 데이터 수집: 야후 파이낸스에서 다운로드 받았다. 미국 주식의 S&P 500 ETF 데이터를 다운로드 받았다.

- 출저:https://finance.yahoo.com/quote/SPY/history?period1=728265600&period2=1700438400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true

In [13]:
#먼저 필요한 라이브러리들을 import 하자.

import pandas as pd
import numpy as np
import datetime #날짜를 계산하기 위해 import 한다.


#데이터를 읽어들이고, 상위 5개의 행을 불러온다.
read_df = pd.read_csv('data/SPY.csv')
read_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,24.941393,1003200
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,25.118782,480500
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,25.171989,201300
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,25.438087,529400
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,25.544533,531500


- 이 전략을 쓰기 위해서는 각 월의 월말 데이터에 접근하자.
- 그 전에, 우리는 종가 데이터만 사용할 것이다. 그러므로 종가 변수만 선택해서 별도의 데이터프레임을 만들자.



In [14]:
Close_df = read_df.loc[:,['Date','Adj Close']].copy()  #데이터를 안전하게 복사하기 위해 copy 매소드를 사용한다.

Close_df.head()

,Date,Adj Close
0,1993-01-29,24.941393
1,1993-02-01,25.118782
2,1993-02-02,25.171989
3,1993-02-03,25.438087
4,1993-02-04,25.544533


In [15]:
#월말 데이터만 사용하는 코드를 작성하자.

Close_df['year_month'] = Close_df['Date'].map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m'))
#strptime은 datetime을 원하는 형식으로 나타나게 하고 strftime은  그 중에서 알맞은 형식으로 데이터를 바꾼다.

Close_df.head()


,Date,Adj Close,year_month
0,1993-01-29,24.941393,1993-01
1,1993-02-01,25.118782,1993-02
2,1993-02-02,25.171989,1993-02
3,1993-02-03,25.438087,1993-02
4,1993-02-04,25.544533,1993-02


- 이제, 월말 종가에 접근하는 데이터프레임을 만들자.

In [16]:
month_list = Close_df['year_month'].unique()
#month의 list를 만들자.

month_list

array(['1993-01', '1993-02', '1993-03', '1993-04', '1993-05', '1993-06',
       '1993-07', '1993-08', '1993-09', '1993-10', '1993-11', '1993-12',
       '1994-01', '1994-02', '1994-03', '1994-04', '1994-05', '1994-06',
       '1994-07', '1994-08', '1994-09', '1994-10', '1994-11', '1994-12',
       '1995-01', '1995-02', '1995-03', '1995-04', '1995-05', '1995-06',
       '1995-07', '1995-08', '1995-09', '1995-10', '1995-11', '1995-12',
       '1996-01', '1996-02', '1996-03', '1996-04', '1996-05', '1996-06',
       '1996-07', '1996-08', '1996-09', '1996-10', '1996-11', '1996-12',
       '1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06',
       '1997-07', '1997-08', '1997-09', '1997-10', '1997-11', '1997-12',
       '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06',
       '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12',
       '1999-01', '1999-02', '1999-03', '1999-04', '1999-05', '1999-06',
       '1999-07', '1999-08', '1999-09', '1999-10', 

In [17]:
month_list_df = pd.DataFrame() #새로운 데이터프레임을 정의한다.
month_last_days =[] #배열을 정의한다.
for m in month_list: #month_list 대로 for 문을 돌린다.
    #기준 연월에 맞는 인덱스의 마지막 날짜 row를 데이터프레임에 추가한다.
    last_day_index  = Close_df[Close_df['year_month'] == m].index[-1]
    #index[-1]을 통해 데이터에 쉽게 접근한다.
     month_last_days.append(Close_df.loc[[last_day_index], :])
    #정의한 배열에 마지막 날의 데이터를 넣는다.
    
#위의 배열의 정보를 dataframe에 삽입한다.
month_list_df = pd.concat(month_last_days)
    

#데이터 프레임의 Date을 인덱스로 삼고, 
month_last_df.set_index(['Date'],inplace=True)

print(month_last_df.head())

IndentationError: unexpected indent (<ipython-input-17-1743ffbd7472>, line 7)

In [18]:
#아래는 위의 오류를 chatgpt로 고친 코드이다.

import pandas as pd

# Assuming you have a DataFrame named Close_df with a 'year_month' column
# and a DataFrame named month_list_df that you want to create

month_list_df = pd.DataFrame()  # Create a new DataFrame

month_last_days = []  # Define an empty list

for m in month_list:  # Iterate through month_list
    # Find the last day index for the given year_month
    last_day_index = Close_df[Close_df['year_month'] == m].index[-1]
    # Append the row corresponding to the last day to the list
    month_last_days.append(Close_df.loc[[last_day_index], :])

# Concatenate the rows in the list to create the new DataFrame
month_list_df = pd.concat(month_last_days)

# Set the 'Date' column as the index
month_list_df.set_index(['Date'], inplace=True)

# Display the first few rows of the resulting DataFrame
print(month_list_df.head())

            Adj Close year_month
Date                            
1993-01-29  24.941393    1993-01
1993-02-26  25.207491    1993-02
1993-03-31  25.772108    1993-03
1993-04-30  25.112654    1993-04
1993-05-28  25.789934    1993-05


# 자, 이제 데이터를 가공하자.

In [19]:
month_list_df['BF_1M_Adj Close'] = month_list_df.shift(1)['Adj Close'] #1개월전 종가 데이터를 가져온다.
month_list_df['BF_1YEAR_Adj Close'] = month_list_df.shift(12)['Adj Close'] #12개월 전 종가 데이터를 가져온다.
month_list_df.fillna(0, inplace=True) #na값을 채운다. 종가를 땡겨오는데, na값이 나는 것은 당연하다.


,Adj Close,year_month,BF_1M_Adj Close,BF_1YEAR_Adj Close
Date,,,,
1993-01-29,24.941393,1993-01,0.000000,0.000000
1993-02-26,25.207491,1993-02,24.941393,0.000000
1993-03-31,25.772108,1993-03,25.207491,0.000000
1993-04-30,25.112654,1993-04,25.772108,0.000000
1993-05-28,25.789934,1993-05,25.112654,0.000000
1993-06-30,25.882965,1993-06,25.789934,0.000000
1993-07-30,25.757311,1993-07,25.882965,0.000000
1993-08-31,26.744524,1993-08,25.757311,0.000000
1993-09-30,26.549948,1993-09,26.744524,0.000000


# 포지션 기록

- 모멘텀 지수를 계산해서 거래가 생길 때 기록할 데이터프레임을 만든다.

In [20]:
position = Close_df.copy()
position.set_index(['Date'],inplace=True) #set_index를 통해 date를 index로 만든다.
position['trade'] = '' #trade부분을 이제부터 채울 것이므로, 빈 데이터로 만든다.
position.head()

,Adj Close,year_month,trade
Date,,,
1993-01-29,24.941393,1993-01,
1993-02-01,25.118782,1993-02,
1993-02-02,25.171989,1993-02,
1993-02-03,25.438087,1993-02,
1993-02-04,25.544533,1993-02,


# 거래 실행

월별 인덱스를 순회하며 1년전 종가 대비 1개월 전 종가 수익률이 얼마인지 계산한다. 수익률이 0 이상이면 모멘텀 현상이 나타난 것으로 판단하는 것이다. 모멘텀 현상이 나타나면, 매수를 한다.

In [21]:
#trading 부분.
ticker = 'SPY'
for x in month_list_df.index: #index를 붙여서, 데이터를 다 불러올 수 있도록 한다.
    signal = ''
    # 절대 모멘텀을 계산한다. 
    momentum = month_list_df.loc[x,'BF_1M_Adj Close'] / month_list_df.loc[x,'BF_1YEAR_Adj Close'] -1
    # 절대 모멘텀 지표 True / False를 판단한다.
    flag = True if ((momentum > 0.0) and (momentum != np.inf) and (momentum != -np.inf))\
    else False \
    and True
    if flag :
        signal = 'buy ' + ticker # 절대 모멘텀 지표가 Positive이면 매수 후 보유.
    print('날짜 : ',x,' 모멘텀 인덱스 : ',momentum, 'flag : ',flag ,'signal : ',signal)
    position.loc[x:,'trade'] = signal

<ipython-input-21-82652c1df13d>:6: RuntimeWarning: invalid value encountered in double_scalars
  momentum = month_list_df.loc[x,'BF_1M_Adj Close'] / month_list_df.loc[x,'BF_1YEAR_Adj Close'] -1
<ipython-input-21-82652c1df13d>:6: RuntimeWarning: divide by zero encountered in double_scalars
  momentum = month_list_df.loc[x,'BF_1M_Adj Close'] / month_list_df.loc[x,'BF_1YEAR_Adj Close'] -1


날짜 :  1993-01-29  모멘텀 인덱스 :  nan flag :  False signal :  
날짜 :  1993-02-26  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-03-31  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-04-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-05-28  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-06-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-07-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-08-31  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-09-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-10-29  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-11-30  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1993-12-31  모멘텀 인덱스 :  inf flag :  False signal :  
날짜 :  1994-01-31  모멘텀 인덱스 :  0.08709204814662908 flag :  True signal :  buy SPY
날짜 :  1994-02-28  모멘텀 인덱스 :  0.11312922813301807 flag :  True signal :  buy SPY
날짜 :  1994-03-31  모멘텀 인덱스 :  0.0569902159342186 flag :  True signal :  buy SPY
날짜 :  1994-04-29  모멘텀 인덱스 :  0.03929465201089477 flag :  True sig

날짜 :  2002-02-28  모멘텀 인덱스 :  -0.07521167690652097 flag :  False signal :  
날짜 :  2002-03-28  모멘텀 인덱스 :  -0.03788515274845494 flag :  False signal :  
날짜 :  2002-04-30  모멘텀 인덱스 :  -0.08412689714683452 flag :  False signal :  
날짜 :  2002-05-31  모멘텀 인덱스 :  -0.13252772505245802 flag :  False signal :  
날짜 :  2002-06-28  모멘텀 인덱스 :  -0.1166243510758268 flag :  False signal :  
날짜 :  2002-07-31  모멘텀 인덱스 :  -0.1733960732590436 flag :  False signal :  
날짜 :  2002-08-30  모멘텀 인덱스 :  -0.1905202523648043 flag :  False signal :  
날짜 :  2002-09-30  모멘텀 인덱스 :  -0.11257511760923766 flag :  False signal :  
날짜 :  2002-10-31  모멘텀 인덱스 :  -0.21583404664514483 flag :  False signal :  
날짜 :  2002-11-29  모멘텀 인덱스 :  -0.2127008343050788 flag :  False signal :  
날짜 :  2002-12-31  모멘텀 인덱스 :  -0.16882691855596665 flag :  False signal :  
날짜 :  2003-01-31  모멘텀 인덱스 :  -0.20808665538265336 flag :  False signal :  
날짜 :  2003-02-28  모멘텀 인덱스 :  -0.2134561674474923 flag :  False signal :  
날짜 :  2003-03-31  모멘텀 인덱스 :  -

날짜 :  2011-04-29  모멘텀 인덱스 :  0.13814708257271424 flag :  True signal :  buy SPY
날짜 :  2011-05-31  모멘텀 인덱스 :  0.27219111349727254 flag :  True signal :  buy SPY
날짜 :  2011-06-30  모멘텀 인덱스 :  0.3265622230803136 flag :  True signal :  buy SPY
날짜 :  2011-07-29  모멘텀 인덱스 :  0.22080160795050996 flag :  True signal :  buy SPY
날짜 :  2011-08-31  모멘텀 인덱스 :  0.2527281851721077 flag :  True signal :  buy SPY
날짜 :  2011-09-30  모멘텀 인덱스 :  0.08655278439055203 flag :  True signal :  buy SPY
날짜 :  2011-10-31  모멘텀 인덱스 :  -0.026082550773489532 flag :  False signal :  
날짜 :  2011-11-30  모멘텀 인덱스 :  0.08021781520706406 flag :  True signal :  buy SPY
날짜 :  2011-12-30  모멘텀 인덱스 :  0.008413468010977754 flag :  True signal :  buy SPY
날짜 :  2012-01-31  모멘텀 인덱스 :  -0.004251214927980307 flag :  False signal :  
날짜 :  2012-02-29  모멘텀 인덱스 :  0.006947142424048369 flag :  True signal :  buy SPY
날짜 :  2012-03-30  모멘텀 인덱스 :  0.050528253110710075 flag :  True signal :  buy SPY
날짜 :  2012-04-30  모멘텀 인덱스 :  0.0537979435863928

In [24]:
position.tail(50)
#오! 좋은 시그널인 것 같다. 계속 사라고 하네?

,Adj Close,year_month,trade
Date,,,
2023-09-11,446.873749,2023-09,buy SPY
2023-09-12,444.422363,2023-09,buy SPY
2023-09-13,444.940552,2023-09,buy SPY
2023-09-14,448.777008,2023-09,buy SPY
2023-09-15,443.369995,2023-09,buy SPY
2023-09-18,443.630005,2023-09,buy SPY
2023-09-19,442.709991,2023-09,buy SPY
2023-09-20,438.640015,2023-09,buy SPY
2023-09-21,431.390015,2023-09,buy SPY


In [25]:
position.tail(25)

,Adj Close,year_month,trade
Date,,,
2023-10-16,436.040009,2023-10,buy SPY
2023-10-17,436.019989,2023-10,buy SPY
2023-10-18,430.209991,2023-10,buy SPY
2023-10-19,426.429993,2023-10,buy SPY
2023-10-20,421.190002,2023-10,buy SPY
2023-10-23,420.459991,2023-10,buy SPY
2023-10-24,423.630005,2023-10,buy SPY
2023-10-25,417.549988,2023-10,buy SPY
2023-10-26,412.549988,2023-10,buy SPY


# 최종: 전략 수익률 확인

전략의 수익률을 확인해보자. 수익률 함수는 다음과 같은 코드로 표현할 수 있다. 매우 중요한 코드이고, 알고리즘에 쓰일 수도 있는 함수이기 때문에 깊은 이해가 필요하다.

In [40]:
#함수의 알고리즘





In [26]:
def returns(book, ticker):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy '+ ticker and book.shift(1).loc[i,'trade'] == '' :     # long 진입
            buy = book.loc[i, 'Adj Close']
            print('진입일 : ',i, 'long 진입가격 : ', buy)
        elif book.loc[i, 'trade'] == 'buy '+ ticker and book.shift(1).loc[i,'trade'] == 'buy '+ ticker :
            # 보유중  
            current = book.loc[i, 'Adj Close']
            rtn = (current - buy) / buy + 1
            book.loc[i, 'return'] = rtn
            
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy '+ticker:     # long 청산
            sell = book.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ',i, 'long 진입가격 : ', buy, ' |  long 청산가격 : ', \
                  sell, ' | return:', round(rtn, 4))
        
        if book.loc[i, 'trade'] == '':     # zero position
            buy = 0.0
            sell = 0.0
            current = 0.0
        
    acc_rtn = 1.0
    for i in book.index:
        if book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy '+ticker:     # long 청산시
            rtn = book.loc[i, 'return']
            acc_rtn = acc_rtn * rtn # 누적수익률 계산
            book.loc[i:, 'acc return'] = acc_rtn

    print ('Accunulated return :', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [27]:
returns(position,ticker)

진입일 :  1994-01-31 long 진입가격 :  28.059195
청산일 :  1994-12-30 long 진입가격 :  28.059195  |  long 청산가격 :  27.221291  | return: 0.9701
진입일 :  1995-02-28 long 진입가격 :  29.28438
청산일 :  2000-12-29 long 진입가격 :  29.28438  |  long 청산가격 :  85.920906  | return: 2.934
진입일 :  2001-02-28 long 진입가격 :  81.180679
청산일 :  2001-03-30 long 진입가격 :  81.180679  |  long 청산가격 :  76.631577  | return: 0.944
진입일 :  2003-07-31 long 진입가격 :  67.485847
청산일 :  2008-02-29 long 진입가격 :  67.485847  |  long 청산가격 :  98.589821  | return: 1.4609
진입일 :  2009-10-30 long 진입가격 :  79.513321
청산일 :  2011-10-31 long 진입가격 :  79.513321  |  long 청산가격 :  100.216187  | return: 1.2604
진입일 :  2011-11-30 long 진입가격 :  99.808922
청산일 :  2012-01-31 long 진입가격 :  99.808922  |  long 청산가격 :  105.528709  | return: 1.0573
진입일 :  2012-02-29 long 진입가격 :  110.10923
청산일 :  2015-10-30 long 진입가격 :  110.10923  |  long 청산가격 :  180.244659  | return: 1.637
진입일 :  2015-11-30 long 진입가격 :  180.903488
청산일 :  2016-02-29 long 진입가격 :  180.903488  |  long 청산가격 :  168.786575  

16.4076

# 결론 및 더 나아가기
자, 1994년부터 2023년까지의 수익률은 16.4076베라는 것을 알 수 있다.
지금부터는 기간을 달리 해서, 수익률을 알아보도록 하자.


In [28]:
#기간을 달리 해서 수익률을 알아보는 것은 다른 주피터 노트북에 있다.